<a href="https://colab.research.google.com/github/Lindarychwalski/ControllableTalkNet/blob/main/Controllable_TalkNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Controllable TalkNet
To run TalkNet, click on Runtime -> Run all. The interface will appear at the bottom of the page when it's ready.

## Instructions

*   Once the notebook is running, click on Files (the folder icon on the left edge).
*   Upload audio clips of a singing or speaking voice by dragging and dropping them onto the sidebar.
*   Click on "Update file list" in the TalkNet interface. Select an audio file from the dropdown, and type what it says into the Transcript box.
*   Select a character, and press Generate. The first line will take a little longer to generate.

## Tips and tricks
*   If you want to use TalkNet as regular text-to-speech system, without any reference audio, tick the "Disable reference audio" checkbox.
*   You can use [ARPABET](http://www.speech.cs.cmu.edu/cgi-bin/cmudict) to override the pronunciation of words, like this: *She made a little bow, then she picked up her {B OW}.*
*   If you're running out of memory generating lines, try to work with shorter clips.
*   The singing models are trained on very little data, and can have a hard time pronouncing certain words. Try experimenting with ARPABET and punctuation.
*   If the voice is off-key, the problem is usually with the extracted pitch. Press "Debug pitch" to listen to it. Reference audio with lots of echo/reverb or background noise, or singers with a very high vocal range can cause issues.
*   If the singing voice sounds strained, try enabling "Change input pitch" and adjusting it up or down a few semitones. If you're remixing a song, remember to pitch-shift your background track as well.

>Maintained by: `justinjohn0306`

>Special thanks to: `Tapiocapioca#6641` and `effusiveperiscope`

In [1]:
#@markdown **Step 1:** Check which GPU you've been allocated.

!nvidia-smi -L
!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-f204c88b-7e30-d913-2b4f-8c0652478673)
Sat Jul 12 12:51:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |       

In [ ]:
#@markdown **Step 2:** Download dependencies.

#@markdown >### Note: The runtime will crash after this cell completes, which is intended.

import nltk
nltk.download('averaged_perceptron_tagger_eng')

import os

custom_lists = [
    #"https://gist.githubusercontent.com/SortAnon/997cda157954a189259c9876fd804e53/raw/example_models.json",
]

!apt-get install sox libsndfile1 ffmpeg

# 3.10 pytorch-lightning fix
!pip install -U torch --index-url https://download.pytorch.org/whl/cu118
!pip install pytorch-lightning==1.6.5
!pip install torchmetrics==0.11.4
# 3.10 fix
!pip install numpy==1.23.5 scipy==1.10.1 librosa==0.8.1
!pip install tensorflow dash==1.21.0 dash-bootstrap-components==0.13.0 jupyter-dash==0.4.0 psola wget unidecode pysptk frozendict torchvision torchaudio torchtext torch_stft kaldiio pydub pyannote.audio g2p_en pesq pystoi crepe resampy ffmpeg-python torchcrepe einops taming-transformers-rom1504==0.0.6 tensorflow-hub
!pip uninstall gdown -y
!pip install git+https://github.com/wkentaro/gdown.git
!python -m pip install git+https://github.com/effusiveperiscope/NeMo.git
if not os.path.exists("hifi-gan"):
    !git clone -q --recursive https://github.com/justinjohn0306/talknet-hifi-gan hifi-gan
!git clone -q https://github.com/effusiveperiscope/ControllableTalkNet
os.chdir("/content/ControllableTalkNet")
!git archive --output=./files.tar --format=tar HEAD
os.chdir("/content")
!tar xf ControllableTalkNet/files.tar
!rm -rf ControllableTalkNet

# 3.10 werkzeug fix
!python -m pip install werkzeug==2.0.0 flask==2.1.2
# pytorch cuda fix???
!pip install -U torch torchtext torchvision torchaudio torch_stft torchcrepe --index-url https://download.pytorch.org/whl/cu121
!pip install librosa==0.8.1

os.chdir("/content/model_lists")
for c in custom_lists:
    !wget "{c}"
os.chdir("/content")


# restart the runtime
os.kill(os.getpid(), 9)


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 35 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libsox3

In [ ]:
# @markdown **Step 3:** **Start GUI**
using_inline = True
import pkg_resources
from pkg_resources import DistributionNotFound, VersionConflict
"""dependencies = [
"tensorflow==2.4.1",
"dash",
"jupyter-dash",
"psola",
"wget",
"unidecode",
"pysptk",
"frozendict",
"torchvision==0.9.1",
"torchaudio==0.8.1",
"torchtext==0.9.1",
"torch_stft",
"kaldiio",
"pydub",
"pyannote.audio",
"g2p_en",
"pesq",
"pystoi",
"crepe",
"resampy",
"ffmpeg-python",
"numpy",
"scipy",
"nemo_toolkit",
"tqdm",
"gdown",
]
pkg_resources.require(dependencies)"""

from controllable_talknet import *
app.run_server(
    mode="inline",
    #dev_tools_ui=True,
    #dev_tools_hot_reload=True,
    threaded=True,
)

In [ ]:
# @markdown **Step 3B:** If the above fails with a 403 error, do the following:
# @markdown * Go to Runtime -> Restart runtime
# @markdown * Run this cell (click the play button)
# @markdown * Click on the googleusercontent.com link to use TalkNet in a separate tab
try:
    using_inline
except:
    using_inline = False
if not using_inline:
    from controllable_talknet import *
    from google.colab.output import eval_js

    print(eval_js("google.colab.kernel.proxyPort(8050)"))
    app.run_server(
        mode="external",
        debug=False,
        #dev_tools_ui=True,
        #dev_tools_hot_reload=True,
        threaded=True,
    )